##setup

In [ ]:
%%capture
!pip install transformers
!pip install shap

In [2]:
#from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
import numpy as np 
import scipy as sp
#import shap
import pickle
import torch
# Device
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
print(torch.cuda.get_device_name(device))

NVIDIA A100-SXM4-40GB


In [ ]:
!cp /content/drive/MyDrive/fake-news-adversarial-benchmark/utils_fake_news.py .
%run utils_fake_news.py

In [3]:
path='/content/drive/MyDrive/fake-news-adversarial-benchmark/'

# Define paths Fake News

dataset='fake_news'
model_path = path+'Model/fn_model'

path_data_raw=path+'Data/Raw/'

path_csv= path_data_raw+'fake_news.csv'
df = pd.read_csv(path_csv)

path_created='/content/drive/MyDrive/fake-news-adversarial-benchmark/data_created/fake_news/'
shap_path=path_created+'shapvalues_1000_fake_news.sav' 
outputs_path=path_created+'original_outputs_fake_news_1000.pt'
csv_path=path_created+'fake_news_1000.csv'
labels_path=path_created+'labels_fake_news.npy'



In [ ]:
# Instantiate the tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained(model_path,  num_labels = 2) 

In [4]:
df_data=df.sample(1000, random_state=42)
df_data.to_csv(csv_path)

In [ ]:
df_encode = encode_dataframe(df_data['statement'], df_data['label'].tolist())


The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).


##shap fake news

In [ ]:
# define a prediction function https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html#nlp_model
def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=512, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val


In [ ]:
label =[0, 1]
explainer = shap.Explainer(f, tokenizer)

In [ ]:
model.to(device)
shap_values = explainer(df_data['statement'], batch_size=20)

Partition explainer: 1001it [36:18,  2.18s/it]


In [ ]:
# Save shap values
pickle.dump(shap_values, open(shap_path, 'wb'))

# Save labels corresponding to SHAP values

np.save(labels_path, df_data['label'])

##fake news original results

In [ ]:
import itertools
def evaluate(bert_dataloader, bert_model):
    # Generate predictions
    outputs = []
    labels = []
    with torch.no_grad():
        for step, batch in enumerate(bert_dataloader):
            # Unpack batch
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            
            # Forward pass
            output = bert_model(b_input_ids, b_input_mask)
            outputs.append(output)   

            # Keep labels
            labels.append(b_labels)     
            
    # Stack outputs
    outputs = torch.vstack([item[0].detach() for item in outputs])

    # Stack labels
    labs = [list(i.cpu().numpy()) for i in labels]
    labs = np.array(list(itertools.chain(*labs)))

    return outputs, labs

In [ ]:
# Load test dataset into dataloader
batch_size = 32

dataloader = torch.utils.data.DataLoader(df_encode, batch_size=batch_size, shuffle=False)
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
# Evaluate
outputs, labs = evaluate(dataloader, model)

In [ ]:
torch.save(outputs,outputs_path)

##checking fake news train vs test


In [ ]:
test = torch.load(path_pt)

In [ ]:
df_train=pd.read_csv(path_data_raw+'fake_news_train.csv')

In [ ]:
len(df_data)

20242

In [ ]:
len(df_train)

20800

In [ ]:
tokenizer.decode(test[0][0])

'[CLS] mark zuckerberg just betrayed hillary ; is this the true reason why ( video ) [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'